<a href="https://colab.research.google.com/github/Tinii1601/Lab4_DS201/blob/main/22521471.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab 4 - bài 1

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_train = pd.read_json('/content/drive/MyDrive/deeplearning/Lab4/Dataset/small-PhoMT/small-train.json')
df_dev = pd.read_json('/content/drive/MyDrive/deeplearning/Lab4/Dataset/small-PhoMT/small-dev.json')
df_test = pd.read_json('/content/drive/MyDrive/deeplearning/Lab4/Dataset/small-PhoMT/small-test.json')

In [4]:
df_train.head()

,english,vietnamese
0,It begins with a countdown .,Câu chuyện bắt đầu với buổi lễ đếm ngược .
1,"On August 14th , 1947 , a woman in Bombay goes...","Ngày 14 , tháng 8 , năm 1947 , gần nửa đêm , ở..."
2,"Across India , people hold their breath for th...","Cùng lúc , trên khắp đất Ấn , người ta nín thở..."
3,"And at the stroke of midnight , a squirming in...","Khi đồng hồ điểm thời khắc nửa đêm , một đứa t..."
4,"These events form the foundation of "" Midnight...","Những sự kiện này là nền móng tạo nên "" Những ..."


In [5]:
print(len(df_train), len(df_dev), len(df_test))
df_train.head()

20000 2000 2000


,english,vietnamese
0,It begins with a countdown .,Câu chuyện bắt đầu với buổi lễ đếm ngược .
1,"On August 14th , 1947 , a woman in Bombay goes...","Ngày 14 , tháng 8 , năm 1947 , gần nửa đêm , ở..."
2,"Across India , people hold their breath for th...","Cùng lúc , trên khắp đất Ấn , người ta nín thở..."
3,"And at the stroke of midnight , a squirming in...","Khi đồng hồ điểm thời khắc nửa đêm , một đứa t..."
4,"These events form the foundation of "" Midnight...","Những sự kiện này là nền móng tạo nên "" Những ..."


In [6]:
import re

def normalize_text(s):
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    return s

df_train["english"] = df_train["english"].map(normalize_text)
df_train["vietnamese"] = df_train["vietnamese"].map(normalize_text)

df_dev["english"] = df_dev["english"].map(normalize_text)
df_dev["vietnamese"] = df_dev["vietnamese"].map(normalize_text)

df_test["english"] = df_test["english"].map(normalize_text)
df_test["vietnamese"] = df_test["vietnamese"].map(normalize_text)


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

VOCAB_EN = 40000
VOCAB_VI = 40000

tokenizer_en = Tokenizer(num_words=VOCAB_EN, oov_token="<unk>")
tokenizer_vi = Tokenizer(num_words=VOCAB_VI, oov_token="<unk>")

tokenizer_en.fit_on_texts(df_train["english"])
tokenizer_vi.fit_on_texts(df_train["vietnamese"])

In [8]:
def percentile_len(texts, q=0.95):
    lens = texts.str.split().str.len()
    return int(lens.quantile(q))

max_len_en = percentile_len(df_train["english"], 0.95)
max_len_vi = percentile_len(df_train["vietnamese"], 0.95) + 2

print(f"Max Len EN: {max_len_en}, Max Len VI: {max_len_vi}")

Max Len EN: 46, Max Len VI: 57


In [9]:
def prepare_data_fast(df, tokenizer_en, tokenizer_vi):
    # Tokenize và Padding toàn bộ English
    en_ids = tokenizer_en.texts_to_sequences(df["english"])
    en_ids = pad_sequences(en_ids, maxlen=max_len_en, padding="post")

    # Tokenize và Padding toàn bộ Vietnamese
    vi_in_texts = df["vietnamese"].apply(lambda x: "<sos> " + x)
    vi_out_texts = df["vietnamese"].apply(lambda x: x + " <eos>")

    vi_in_ids = tokenizer_vi.texts_to_sequences(vi_in_texts)
    vi_out_ids = tokenizer_vi.texts_to_sequences(vi_out_texts)

    vi_in = pad_sequences(vi_in_ids, maxlen=max_len_vi, padding="post")
    vi_out = pad_sequences(vi_out_ids, maxlen=max_len_vi, padding="post")

    return (en_ids.astype("int32"), vi_in.astype("int32")), vi_out.astype("int32")

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_input, train_target = prepare_data_fast(df_train, tokenizer_en, tokenizer_vi)
dev_input, dev_target = prepare_data_fast(df_dev, tokenizer_en, tokenizer_vi)
test_input, test_target = prepare_data_fast(df_test, tokenizer_en, tokenizer_vi)

In [11]:
from tensorflow.keras import backend as K
K.clear_session()

In [12]:
BATCH_SIZE = 64

train_ds = tf.data.Dataset.from_tensor_slices((train_input, train_target)) \
            .shuffle(10000).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

dev_ds = tf.data.Dataset.from_tensor_slices((dev_input, dev_target)) \
            .batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_input, test_target)) \
            .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [13]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")


In [14]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

EMB = 300
HID = 256
LAYERS = 5

# Encoder
enc_in = Input(shape=(max_len_en,))
x = Embedding(VOCAB_EN, EMB, mask_zero=True)(enc_in)

enc_states = []
for i in range(LAYERS):
    x, h, c = LSTM(HID, return_sequences=True, return_state=True)(x)
    enc_states.append((h, c))

# Decoder
dec_in = Input(shape=(max_len_vi,))
y = Embedding(VOCAB_VI, EMB, mask_zero=True)(dec_in)

for i in range(LAYERS):
    y, _, _ = LSTM(
        HID,
        return_sequences=True,
        return_state=True,
        name=f"dec_lstm_{i+1}"
    )(
        y,
        initial_state=enc_states[i]
    )

dec_out = Dense(VOCAB_VI, activation="softmax")(y)

model = Model([enc_in, dec_in], dec_out)


In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy"
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 46)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 57)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 46, 300)   │ 12,000,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 46)        │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 57, 300)   │ 12,000,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 46, 256), │    570,368 │ embedding[0][0],  │
│                     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dec_lstm_1 (LSTM)   │ [(None, 57, 256), │    570,368 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm[0][1],       │
│                     │ (None, 256)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 46, 256), │    525,312 │ lstm[0][0],       │
│                     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dec_lstm_2 (LSTM)   │ [(None, 57, 256), │    525,312 │ dec_lstm_1[0][0], │
│                     │ (None, 256),      │            │ lstm_1[0][1],     │
│                     │ (None, 256)]      │            │ lstm_1[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 46, 256), │    525,312 │ lstm_1[0][0],     │
│                     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dec_lstm_3 (LSTM)   │ [(None, 57, 256), │    525,312 │ dec_lstm_2[0][0], │
│                     │ (None, 256),      │            │ lstm_2[0][1],     │
│                     │ (None, 256)]      │            │ lstm_2[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 46, 256), │    525,312 │ lstm_2[0][0],     │
│                     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dec_lstm_4 (LSTM)   │ [(None, 57, 256), │    525,312 │ dec_lstm_3[0][0], │
│                     │ (None, 256),      │            │ lstm_3[0][1],     │
│                     │ (None, 256)]      │            │ lstm_3[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 46, 256), │    525,312 │ lstm_3[0][0],     │
│                     │ (None, 256),      │            │ not_equal[0][0] 

 Total params: 39,623,232 (151.15 MB)

 Trainable params: 39,623,232 (151.15 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
num_train = len(df_train)
num_dev   = len(df_dev)

steps_per_epoch = len(df_train) // BATCH_SIZE
val_steps = len(df_dev) // BATCH_SIZE

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

ckpt = ModelCheckpoint(
    "seq2seq_best.weights.h5",
    save_best_only=True,
    save_weights_only=True
)

early = EarlyStopping(patience=3, restore_best_weights=True)


In [18]:
model.fit(
    train_ds,
    validation_data=dev_ds,
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_steps=val_steps,
    callbacks=[ckpt, early]
)

Epoch 1/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 91s 251ms/step - loss: 7.3748 - val_loss: 6.4781
Epoch 2/50
  1/312 ━━━━━━━━━━━━━━━━━━━━ 51s 164ms/step - loss: 6.2699

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


312/312 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 6.2699 - val_loss: 6.4784
Epoch 3/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 129s 231ms/step - loss: 6.3856 - val_loss: 6.4525
Epoch 4/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 6.2137 - val_loss: 6.4530
Epoch 5/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 80s 232ms/step - loss: 6.3494 - val_loss: 6.4417
Epoch 6/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 6.1855 - val_loss: 6.4426
Epoch 7/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 72s 230ms/step - loss: 6.3306 - val_loss: 6.4256
Epoch 8/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - loss: 6.1457 - val_loss: 6.4256
Epoch 9/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 72s 230ms/step - loss: 6.2864 - val_loss: 6.3336
Epoch 10/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6.0014 - val_loss: 6.3346
Epoch 11/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 72s 229ms/step - loss: 6.1450 - val_loss: 6.1500
Epoch 12/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 5.7450 - val_loss: 6.1476
Epoch 13/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 71s

In [19]:
import tensorflow as tf
import numpy as np
import re
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score

smooth = SmoothingFunction().method1


In [20]:
def greedy_decode_batch(model, enc_in, max_len):
    B = enc_in.shape[0]

    # decoder input luôn có shape (B, max_len)
    dec_in = tf.zeros((B, max_len), dtype=tf.int32)

    outputs = []

    for t in range(max_len):
        logits = model([enc_in, dec_in], training=False)

        # lấy token tại vị trí t
        next_id = tf.argmax(logits[:, t, :], axis=-1, output_type=tf.int32)

        outputs.append(next_id)

        # ghi token vào dec_in tại vị trí t
        dec_in = tf.tensor_scatter_nd_update(
            dec_in,
            indices=tf.stack(
                [tf.range(B), tf.fill([B], t)], axis=1
            ),
            updates=next_id
        )

    return tf.stack(outputs, axis=1)  # (B, max_len)


In [21]:
y_true_texts = []
y_pred_texts = []

for (enc_in, _), dec_out in test_ds:
    preds = greedy_decode_batch(model, enc_in, max_len_vi)

    for gt, pr in zip(dec_out.numpy(), preds.numpy()):
        gt = [x for x in gt if x != 0]
        pr = [x for x in pr if x != 0]

        y_true_texts.append(tokenizer_vi.sequences_to_texts([gt])[0])
        y_pred_texts.append(tokenizer_vi.sequences_to_texts([pr])[0])


In [22]:
def norm(s):
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    return s

refs = [norm(x) for x in y_true_texts]
hyps = [norm(x) for x in y_pred_texts]

In [23]:
def bleu_scores(refs, hyps):
    b1=b2=b3=b4=0
    for r,h in zip(refs,hyps):
        r,h = r.split(), h.split()
        b1+=sentence_bleu([r],h,(1,0,0,0),smooth)
        b2+=sentence_bleu([r],h,(.5,.5,0,0),smooth)
        b3+=sentence_bleu([r],h,(1/3,)*3+(0,),smooth)
        b4+=sentence_bleu([r],h,(.25,)*4,smooth)
    n=len(refs)
    return b1/n,b2/n,b3/n,b4/n


In [24]:
def lcs(a,b):
    dp=[[0]*(len(b)+1) for _ in range(len(a)+1)]
    for i in range(len(a)):
        for j in range(len(b)):
            dp[i+1][j+1]=dp[i][j]+1 if a[i]==b[j] else max(dp[i][j+1],dp[i+1][j])
    return dp[-1][-1]

def rouge_l(refs, hyps):
    s=0
    for r,h in zip(refs,hyps):
        r,h=r.split(),h.split()
        l=lcs(r,h)
        p=l/max(len(h),1); r_=l/max(len(r),1)
        s+=0 if p+r_==0 else 2*p*r_/(p+r_)
    return s/len(refs)


In [25]:
b1,b2,b3,b4 = bleu_scores(refs,hyps)
print(f"BLEU@1 : {b1:.4f}")
print(f"BLEU@2 : {b2:.4f}")
print(f"BLEU@3 : {b3:.4f}")
print(f"BLEU@4 : {b4:.4f}")
print(f"ROUGE-L: {rouge_l(refs,hyps):.4f}")


BLEU@1 : 0.0729
BLEU@2 : 0.0172
BLEU@3 : 0.0079
BLEU@4 : 0.0054
ROUGE-L: 0.0829
